In [1]:
import eli5
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from math import exp
from sklearn.pipeline import make_pipeline

In [2]:
X_train = ["new york is a hell of a town",
            "new york was originally dutch",
            "the big apple is great",
            "new york is also called the big apple",
            "nyc is nice",
            "people abbreviate new york city as nyc",
            "the capital of great britain is london",
            "london is in the uk",
            "london is in england",
            "london is in great britain",
            "it rains a lot in london",
            "london hosts the british museum",
            "new york is great and so is london",
            "i like london better than new york"]
y_train = [['New York'],['New York'],['New York'],['New York'],['New York'],['New York'],['London'],['London'],['London'],['London'],['London'],['London'],['New York','London'],['New York','London']]

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)

X_test = ['nice day in nyc', 'welcome to london', 'hello simon welcome to new york. enjoy it here and london too']

vec = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=15)
clf = OneVsRestClassifier(LogisticRegressionCV())
pipeline = make_pipeline(vec, clf)

pipeline.fit(X_train, y_train)
predicted = pipeline.predict(X_test)
for item, labels in zip(X_test, mlb.inverse_transform(predicted)):
    print('%s => %s' % (item, labels))

nice day in nyc => ('New York',)
welcome to london => ('London',)
hello simon welcome to new york. enjoy it here and london too => ('London', 'New York')


In [3]:
print(pipeline.decision_function(X_test))

[[ -7.96273351   1.04803743]
 [ 22.19686347  -1.39109585]
 [  5.48828931   1.28660432]]


In [4]:
print(np.vectorize(lambda x: 1 / (1 + exp(-x)))(pipeline.decision_function(X_test)))

[[  3.48078806e-04   7.40397853e-01]
 [  1.00000000e+00   1.99232868e-01]
 [  9.95882115e-01   7.83571880e-01]]


In [5]:
eli5.show_prediction(clf, X_test[0], vec=vec, target_names=mlb.classes_)

Weight,Feature
-4.820,Highlighted in text (sum)
Weight,Feature
+0.846,Highlighted in text (sum)


In [6]:
eli5.show_prediction(clf, X_test[1], vec=vec, target_names=mlb.classes_)

Weight,Feature
+25.340,Highlighted in text (sum)
Weight,Feature
-1.594,Highlighted in text (sum)


In [7]:
eli5.show_prediction(clf, X_test[2], vec=vec, target_names=mlb.classes_)

Weight,Feature
+8.631,Highlighted in text (sum)
Weight,Feature
+1.084,Highlighted in text (sum)


In [8]:
setattr(clf.estimator, 'classes_', clf.classes_)
setattr(clf.estimator, 'coef_', clf.coef_)
setattr(clf.estimator, 'intercept_', clf.intercept_)
eli5.show_weights(clf.estimator, vec=vec, target_names=mlb.classes_)

y=London top features,y=New York top features
Weight,Feature
+25.340,london
+4.706,great
+2.079,lot london
+2.079,lot
+1.266,great britain
+1.266,britain
+0.896,museum
-1.682,new
-1.682,new york
